# Proyecto X

## Lectura de los datos y exploración de los encabezados de los archivos.

In [1]:
import pandas as pd

In [2]:
geolocation = pd.read_csv('/content/Copia de ecommerce_geolocation_dataset.csv')
customers = pd.read_csv('/content/Copia de ecommerce_customers_dataset.csv')
order_items = pd.read_csv('/content/Copia de ecommerce_order_items_dataset.csv')
order_payments = pd.read_csv('/content/Copia de ecommerce_order_payments_dataset.csv')
order_reviews = pd.read_csv('/content/Copia de ecommerce_order_reviews_dataset.csv')
orders = pd.read_csv('/content/Copia de ecommerce_orders_dataset.csv')
products = pd.read_csv('/content/Copia de ecommerce_products_dataset.csv')
sellers = pd.read_csv('/content/Copia de ecommerce_sellers_dataset.csv')
category_names = pd.read_csv('/content/Copia de product_category_name_translation.csv')

In [3]:
print(geolocation.columns.tolist())

['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']


In [4]:
print(customers.columns.tolist())

['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']


In [5]:
print(order_items.columns.tolist())

['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']


In [6]:
print(order_payments.columns.tolist())

['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']


In [7]:
print(order_reviews.columns.tolist())

['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']


In [8]:
print(orders.columns.tolist())

['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']


In [9]:
print(products.columns.tolist())

['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']


In [10]:
print(sellers.columns.tolist())

['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']


In [11]:
print(category_names.columns.tolist())

['product_category_name', 'product_category_name_english']


## BBT1-14 Creación de la base de datos y carga de datos

Crear una conexión a la base de datos de SQLite llamada ecommerce.db Importar los archivos y guardar los DataFrames en la base de datos como tablas:

ecommerce_customers_dataset.csv

ecommerce_order_items_dataset.csv

ecommerce_order_payments_dataset.csv

ecommerce_order_reviews_dataset.csv

ecommerce_orders_dataset.csv

ecommerce_products_dataset.csv

ecommerce_sellers_dataset.csv

product_category_name_translation.csv

In [ ]:
import sqlite3 as sql

#Conexion a la base de datos
conn = sql.connect('ecommerce.db')
cursor = conn.cursor()

#script de creacion de tablas
script = '''
CREATE TABLE products (
    product_id TEXT PRIMARY KEY,
    product_category_name TEXT,
    product_name_lenght REAL,
    product_description_lenght REAL,
    product_photos_qty REAL,
    product_weight_g REAL,
    product_length_cm REAL,
    product_height_cm REAL,
    product_width_cm REAL
);
CREATE TABLE order_items (
    order_id TEXT,
    order_item_id INTEGER,
    product_id TEXT,
    seller_id TEXT,
    shipping_limit_date TEXT,
    price REAL,
    freight_value REAL,
    PRIMARY KEY (order_id, order_item_id)
);
CREATE TABLE order_payments (
    order_id TEXT,
    payment_sequential INTEGER,
    payment_type TEXT,
    payment_installments INTEGER,
    payment_value REAL,
    PRIMARY KEY(order_id,payment_sequential)
);
CREATE TABLE orders (
    order_id TEXT PRIMARY KEY,
    customer_id TEXT,
    order_status TEXT,
    order_purchase_timestamp TEXT,
    order_approved_at TEXT,
    order_delivered_carrier_date TEXT,
    order_delivered_customer_date TEXT,
    order_estimated_delivery_date TEXT
);
CREATE TABLE customers (
    customer_id TEXT PRIMARY KEY,
    customer_unique_id TEXT,
    customer_zip_code_prefix INTEGER,
    customer_city TEXT,
    customer_state TEXT
);
CREATE TABLE geolocation(
  geolocation_zip_code_prefix INTEGER PRIMARY KEY ,
  geolocation_lat REAL,
  geolocation_lng REAL,
  geolocation_city TEXT,
  geolocation_state TEXT
);
CREATE TABLE order_review(
  review_id TEXT,
  order_id TEXT,
  review_score INT,
  review_comment_title TEXT,
  review_comment_message TEXT,
  review_creation_date TEXT,
  review_answer_timestamp TEXT
  PRIMARY KEY(order_id,review_id)
);
CREATE TABLE sellers(
  seller_id TEXT PRIMARY KEY,
  seller_zip_code_prefix INTEGER,
  seller_city TEXT,
  seller_state TEXT
);
CREATE TABLE category(
  product_category_name TEXT,
  product_category_name_english TEXT
);
'''
cursor.executescript(script)
conn.commit()


Inserción de los datos de los csv a la base de datos

In [13]:
geolocation.to_sql('geolocation',conn, index=True, if_exists='replace')
customers.to_sql('customers',conn, index=True, if_exists='replace')
order_items.to_sql('order_items',conn, index=True, if_exists = 'replace')
order_payments.to_sql('order_payments',conn, index=True, if_exists = 'replace')
order_reviews.to_sql('order_review',conn, index=True, if_exists = 'replace')
orders.to_sql('orders',conn, index=True, if_exists = 'replace')
products.to_sql('products',conn, index=True, if_exists = 'replace')
sellers.to_sql('sellers',conn, index=True, if_exists = 'replace')
category_names.to_sql('category',conn, index=True, if_exists = 'replace')

71

**Creación de tabla geolocation2 con latitudes y longitudes promedio.**

Con el fin de evitar la duplicidad de datos usando el criterio de promediar latitudes y longitudes agrupando por código postal.

In [16]:
query_geo = '''
  SELECT geolocation_zip_code_prefix, AVG(geolocation_lat) as promedio_latitud, AVG(geolocation_lng) as promedio_longitud, geolocation_city, geolocation_state
  FROM geolocation
  GROUP BY geolocation_zip_code_prefix
  ORDER BY geolocation_zip_code_prefix;
  '''
geo_2 = pd.read_sql_query(query_geo, conn)

geo_2.set_index('geolocation_zip_code_prefix', inplace=True)

In [17]:
geo_2.to_sql('geolocation2', conn, index=True, if_exists='replace')
geo_2.head(5)

,promedio_latitud,promedio_longitud,geolocation_city,geolocation_state
geolocation_zip_code_prefix,,,,
1001,-23.550190,-46.634024,sao paulo,SP
1002,-23.548146,-46.634979,sao paulo,SP
1003,-23.548994,-46.635731,sao paulo,SP
1004,-23.549799,-46.634757,sao paulo,SP
1005,-23.549456,-46.636733,sao paulo,SP


## BBT1-21  Extracción de datos desde las tablas orders_items y products

Crear un DataFrame que contenga el JOIN de la tabla orders_items (oi) y tabla products (p).  
Extraer los datos de items con categorias

In [18]:
BBT1_21_query = '''
SELECT oi.*, p.product_category_name
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
ORDER BY p.product_category_name ASC;
'''

BBT1_21 = pd.read_sql_query(BBT1_21_query, conn)
BBT1_21.head(5)

,index,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name
0,123,0046e1d57f4c07c8c92ab26be8c3dfc0,1,ff6caf9340512b8bf6d2a2a6df032cfa,38e6dada03429a47197d5d584d793b41,2017-10-02 15:49:17,7.79,7.78,None
1,125,00482f2670787292280e0a8153d82467,1,a9c404971d1a5b1cbc2e4070e02731fd,702835e4b785b67a084280efca355756,2017-02-17 16:18:07,7.60,10.96,None
2,132,004f5d8f238e8908e6864b874eda3391,1,5a848e4ab52fd5445cdc07aab1c40e48,c826c40d7b19f62a09e2d7c5e7295ee2,2018-03-06 09:29:25,122.99,15.61,None
3,142,0057199db02d1a5ef41bacbf41f8f63b,1,41eee23c25f7a574dfaf8d5c151dbb12,e5a3438891c0bfdb9394643f95273d8e,2018-01-25 09:07:51,20.30,16.79,None
4,171,006cb7cafc99b29548d4f412c7f9f493,1,e10758160da97891c2fdcbc35f0f031d,323ce52b5b81df2cd804b017b7f09aa7,2018-02-22 13:35:28,56.00,14.14,None


## BBT1-22 Extracción de datos desde las tablas order y payments

Crear un DataFrame que contenga el JOIN de la tabla orders (o) y tabla payments (op). Extraer los datos de órdenes con sus pagos asociados.

In [26]:
BBT1_22_query = '''
  SELECT o.customer_id, o.order_id, o.order_status, op.payment_type, op.payment_value,
         op.payment_sequential, op.payment_installments, o.order_purchase_timestamp, o.order_approved_at,
         o.order_delivered_carrier_date, o.order_delivered_customer_date, o.order_estimated_delivery_date
  FROM orders o
  JOIN order_payments op ON o.order_id = op.order_id
  ;
'''

BBT1_22 = pd.read_sql_query(BBT1_22_query, conn)
BBT1_22.head(5)

,customer_id,order_id,order_status,payment_type,payment_value,payment_sequential,payment_installments,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,delivered,credit_card,18.12,1,1,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,delivered,voucher,18.59,2,1,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
2,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7,delivered,voucher,2.00,3,1,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
3,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451,delivered,boleto,141.46,1,1,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
4,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d,delivered,credit_card,179.12,1,3,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


## BBT1-31 Filtrar datos, reducir volumen de orders_items y products

Filtrar el DataFrame de las categorías e ítems para resguardar los 3 ítems de mayor importe.   
PARA: Reducir el volumen de datos y trabajar solo con los ítems que mayor ingreso generan por categoría.

In [20]:
#Guardar el DF en la base de datos como tabla
BBT1_21.to_sql('Query_BBT1_21', conn, if_exists='replace', index=True)

112650

In [21]:
#Filtrar el DataFrame de las categorías e ítems para resguardar los 3 ítems de mayor importe por categoria

BBT1_31_query = '''
SELECT product_id, product_category_name, price, freight_value
FROM (
  SELECT product_id, product_category_name, price, freight_value,
         ROW_NUMBER() OVER (PARTITION BY product_category_name ORDER BY price DESC) AS rn
  FROM Query_BBT1_21
) ranked
  WHERE rn <=3
  ORDER BY product_category_name ASC;
'''

BBT1_31 = pd.read_sql_query(BBT1_31_query, conn)
BBT1_31.head(9)

,product_id,product_category_name,price,freight_value
0,4c50dcc50f1512f46096d6ef0142c4a9,None,3980.00,62.74
1,0e030462875259ec0cb868f7ecf1fd5e,None,2740.00,54.50
2,eed5cbd74fac3bd79b7c7ec95fa7507d,None,1989.00,34.58
3,2b69866f22de8dad69c976771daba91c,agro_industria_e_comercio,2990.00,194.55
4,b7a60a397d4efd05c1b5d398fb9f9097,agro_industria_e_comercio,2399.00,68.33
5,cd2f5c10e4e8dbc701f0bb68a09fdfe8,agro_industria_e_comercio,2199.00,116.83
6,21b4d9c7183dd6f20f584f35c09bd361,alimentos,274.99,10.74
7,618c65cb5359281b2c830b9f3f2e1f36,alimentos,219.00,19.58
8,53d71d8d9d6878f6f0b9fe0ed79445d5,alimentos,210.00,16.57


## BBT1-32 Filtrar datos, reducir volumen de order y payments

Filtrar el DataFrame de órdenes y pagos para resguardar los 3 pagos de mayor importe por cliente.   
PARA: Reducir el volumen de datos y trabajar solo con los clientes que generan mas ingreso y analizar sus métodos de pagos.

In [22]:
#Guardar el DF en la base de datos como tabla
BBT1_22.to_sql('Query_BBT1_22', conn, if_exists='replace', index=True)

103886

A fin de analizar la ordenes de mayor valor, y poder a su vez analizar sus métodos de pago, se crea una CTE que sume los pagos parciales por método de pago de cada orden, para poder en primera instancia reconocer las ordenes de mayor valor y luego analizar luego sus métodos de pago.
Dado que los clientes que generaron mayores ingresos no realizaron más de 3 pagos, se limitan a 20 los registros para analisis.

In [23]:
#Filtrar el DataFrame de ordenes y pagos para resguardar las ordenes que generaron mayores ingresos.

BBT1_32_query = '''
WITH OrderTotal AS (
    SELECT order_id, sum(payment_value) as total_price
    FROM Query_BBT1_22
    GROUP BY order_id
  )

  SELECT q.customer_id, q.order_id, q.order_status, q.payment_type, q.payment_value, ot.total_price
  FROM Query_BBT1_22 q
  JOIN OrderTotal ot ON ot.order_id = q.order_id
  ORDER BY total_price DESC, customer_id
  LIMIT 20
  ;

'''

BBT1_32 = pd.read_sql_query(BBT1_32_query, conn)
BBT1_32

,customer_id,order_id,order_status,payment_type,payment_value,total_price
0,1617b1357756262bfa56ab541c47bc16,03caa2c082116e1d31e67e9ae3700499,delivered,credit_card,13664.08,13664.08
1,ec5b2ba62e574342386871631fafd3fc,736e1922ae60d0d6a89247b851902527,delivered,boleto,7274.88,7274.88
2,c6e2731c5b391845f6800c97401a43a9,0812eb902a67711a1cb742b3cdaa65ae,delivered,credit_card,6929.31,6929.31
3,f48d464a0baaea338cb25f816991ab1f,fefacc66af859508bf1a7934eab1e97f,delivered,boleto,6922.21,6922.21
4,3fd6777bbce08a352fddd04e4a7cc8f6,f5136e38d1a14a4dbd87dff67da82701,delivered,boleto,6726.66,6726.66
5,05455dfa7cd02f13d132aa7a6a9729c6,2cc9089445046817a7539d90805e6e5a,delivered,boleto,6081.54,6081.54
6,df55c14d1476a9a3467f131269c2477f,a96610ab360d42a2e5335a3998b4718a,delivered,credit_card,4950.34,4950.34
7,e0a2412720e9ea4f26c1ac985f6a7358,b4c4b76c642808cbe472a32b86cddc95,canceled,credit_card,4809.44,4809.44
8,24bbf5fd2f2e1b359ee7de94defc4a15,199af31afc78c699f0dbf71fb178d4d4,delivered,credit_card,4764.34,4764.34
9,3d979689f636322c62418b6346b1c6d2,8dbc85d1447242f3b127dda390d56e19,delivered,credit_card,4681.78,4681.78


In [24]:
#Guardar el DF en la base de datos como tabla para analizar
BBT1_32.to_sql('Query_BBT1_32', conn, if_exists='replace', index=True)

20

Se genera segundo filtro sobre el DF generado en esta story para analizar los métodos de pago de las ordenes que generaron mayores ingresos.

In [25]:
BBT1_32_query_bis = '''
SELECT payment_type, sum(payment_value) AS ingreso_total, count(payment_type) as cant_transacciones
FROM Query_BBT1_32
GROUP BY payment_type
ORDER BY ingreso_total DESC
;
'''

BBT1_32_bis = pd.read_sql_query(BBT1_32_query_bis, conn)
BBT1_32_bis

,payment_type,ingreso_total,cant_transacciones
0,credit_card,64776.72,14
1,boleto,27005.29,4
2,debit_card,8609.01,2


## BBT1-41 Carga de datos, generacion de archivos .csv de orders_items y products

Generar archivos .csv a partir del Dataframe generado de orders_items y products.
Los archivos deben cumplir los siguientes requerimientos:
Contener cabecera.
Separador “,”.
Ser guardados en un directorio llamado Results\Analysis_D.
PARA: Resguardar los datos enriquecidos, según requerimiento.

**Se omite la creación del directorio y se asume que el directorio especificado ya existe:**

In [27]:
#Nombre del archivo.csv
csv_filename = 'BBT1_31.csv'

# Ruta completa del archivo CSV
csv_path = 'Results/Analysis_D/' + csv_filename

BBT1_31.to_csv('BBT1_31.csv', index=False, sep=',', header=True)

print(f'Archivo CSV guardado en: {csv_path}')

Archivo CSV guardado en: Results/Analysis_D/BBT1_31.csv


## BBT1-42 Carga de datos, generacion de archivos .csv  de order y payments

Generar archivos .csv a partir del Dataframe generado de orders y payments.
Los archivos deben cumplir los siguientes requerimientos:
Contener cabecera.
Separador “,”.
Ser guardados en un directorio llamado Results\Analysis_E.
PARA: Resguardar los datos enriquecidos, según requerimiento.

**Se omite la creación del directorio y se asume que el directorio especificado ya existe:**

In [28]:
#Nombre del archivo.csv
csv_filename = 'BBT1_32.csv'

# Ruta completa del archivo CSV
csv_path = 'Results/Analysis_E/' + csv_filename

BBT1_32.to_csv('BBT1_32.csv', index=False, sep=',', header=True)

print(f'Archivo CSV guardado en: {csv_path}')

Archivo CSV guardado en: Results/Analysis_E/BBT1_32.csv


Se realiza el guardado de la segunda consulta del filtrado tambien.

In [29]:
#Nombre del archivo.csv
csv_filename = 'BBT1_32_bis.csv'

# Ruta completa del archivo CSV
csv_path = 'Results/Analysis_E/' + csv_filename

BBT1_32_bis.to_csv('BBT1_32_bis.csv', index=False, sep=',', header=True)

print(f'Archivo CSV guardado en: {csv_path}')

Archivo CSV guardado en: Results/Analysis_E/BBT1_32_bis.csv


## BBT1-51 Extraccion de datos desde .csv, análisis D

QUIERO: Cargar los datos del análisis D generados en un DataFrame.
PARA: Poder analizar el conjunto de datos.

In [ ]:
!pip install pyspark

In [31]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [33]:
path_bbt1_31 = '.Results/Analysis_D/BBT1-31.csv'
bbt1_21_spark = spark.read.load(path_bbt1_31, format='csv', sep=',', inferSchema='true', header='true')
bbt1_21_spark.show(5)

+--------------------+---------------------+------+-------------+
|          product_id|product_category_name| price|freight_value|
+--------------------+---------------------+------+-------------+
|4c50dcc50f1512f46...|                 null|3980.0|        62.74|
|0e030462875259ec0...|                 null|2740.0|         54.5|
|eed5cbd74fac3bd79...|                 null|1989.0|        34.58|
|2b69866f22de8dad6...| agro_industria_e_...|2990.0|       194.55|
|b7a60a397d4efd05c...| agro_industria_e_...|2399.0|        68.33|
+--------------------+---------------------+------+-------------+
only showing top 5 rows



## BBT1-52 Extraccion de datos desde .csv, análisis E

QUIERO: Cargar los datos del análisis E generados en un DataFrame.
PARA: Poder analizar el conjunto de datos.

In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [35]:
path_bbt1_32 = '.Results/Analysis_E/BBT1_32.csv'
bbt1_32_spark = spark.read.load(path_bbt1_32, format='csv', sep=',', inferSchema='true', header='true')
bbt1_32_spark.show(5)

+--------------------+--------------------+------------+------------+-------------+-----------+
|         customer_id|            order_id|order_status|payment_type|payment_value|total_price|
+--------------------+--------------------+------------+------------+-------------+-----------+
|1617b1357756262bf...|03caa2c082116e1d3...|   delivered| credit_card|     13664.08|   13664.08|
|ec5b2ba62e5743423...|736e1922ae60d0d6a...|   delivered|      boleto|      7274.88|    7274.88|
|c6e2731c5b391845f...|0812eb902a67711a1...|   delivered| credit_card|      6929.31|    6929.31|
|f48d464a0baaea338...|fefacc66af859508b...|   delivered|      boleto|      6922.21|    6922.21|
|3fd6777bbce08a352...|f5136e38d1a14a4db...|   delivered|      boleto|      6726.66|    6726.66|
+--------------------+--------------------+------------+------------+-------------+-----------+
only showing top 5 rows



Extraccion de datos desde segunda tabla creada anteriormente

In [36]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [37]:
path_bbt1_32_bis = '.Results/Analysis_E/BBT1_32_bis.csv'
bbt1_32_bis_spark = spark.read.load(path_bbt1_32_bis, format='csv', sep=',', inferSchema='true', header='true')
bbt1_32_bis_spark.show(5)

+------------+-----------------+------------------+
|payment_type|    ingreso_total|cant_transacciones|
+------------+-----------------+------------------+
| credit_card|64776.71999999999|                14|
|      boleto|         27005.29|                 4|
|  debit_card|          8609.01|                 2|
+------------+-----------------+------------------+

